In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from collections import defaultdict
from datetime import date
import json

## Scraping Spanish Newspaper: El Mundo

This piece of code scrapes newspaper articles from El Mundo (Spanish newspaper).
It scrapes the newspaper articles and stores into a list.
Then, this list is saved in a json file. 

In [2]:
url = 'https://www.elmundo.es/'
f = urlopen(url)
html_elmundo = BeautifulSoup(f, 'lxml')
f.close()

In [9]:
def scraping_el_mundo(url, max_num_articles = 1000):
    '''
    This function scrapes the Spanish newspaper El Mundo and returns a dictionary which contains information about the scraped articles. 
    ----------------------------------------------------
    Arguments: url (str) - url of the newspaper
               max_num_articles - maximum number of articles to scrape. set to 1000 by default.
    Returns:   el_mundo_articles (dict) - contains title, source, genre, content and level of the article. 
    '''
    f = urlopen(url)
    html_elmundo = BeautifulSoup(f, 'lxml')
    count = 0
    el_mundo_articles = []
    
    for node in html_elmundo.find_all('a'):
        if len(node.attrs.keys()) == 1 and 'href' in node.attrs.keys() and node.attrs['href'].startswith('https:') and node.attrs['href'].endswith('.html'):
            address = node.attrs['href'].split('/')
            genre = 'news-article-' + address[3]
        
            child_url = node.attrs['href']
            f_child = urlopen(child_url)
            html_child = BeautifulSoup(f_child, 'lxml')
        
            for node_in_child in html_child.find_all("script"):
                if 'type' in node_in_child.attrs and node_in_child.attrs['type'] == 'application/ld+json':
                    
                    for script_ in node_in_child.contents:
                        try: 
                            jsonStr = script_.strip()
                            jsonStr = jsonStr.replace('\n', '')    # remove newline
                            jsonList = json.loads(jsonStr)         # dictionary
                        
                            if type(jsonList) == dict and 'articleBody' in jsonList.keys():
                                article_dict = {}
                                article_dict['title']=' '.join(address[4:-1])
                                article_dict['source']=url
                                article_dict['genre']='news-article-' + genre
                                article_dict['content']=jsonList['articleBody']
                                article_dict['level']='B2'
                                count +=1
                                el_mundo_articles.append(article_dict)
                        except:
                            pass
            f_child.close()
            
            if count % 100 == 0 and count > 0:
                print(f'scraped {count} articles')
            if count % 1000 == 0 and count > 0:
                time.sleep(1)
            if count == max_num_articles:
                break
    f.close()
    return el_mundo_articles

In [10]:
el_mundo_articles = scraping_el_mundo(url)
today = date.today()
today_str = today.strftime("%b-%d-%Y")

## Store the data in el_mundo.json file for the articles

In [11]:
filename = '../corpus/newspapers/el_mundo-'+today_str+'.json'
with open(filename, 'w', encoding = 'utf-8') as outfile:
        json.dump(el_mundo_articles, outfile)

## Read the el_mundo.json file 

In [214]:
with open(filename, encoding = 'utf-8') as json_file:
    data = json.load(json_file)

In [217]:
print(data.keys())
print(len(data['content']))

dict_keys(['title', 'source', 'genre', 'content', 'level'])
13


In [216]:
print(f'title: {data["title"][0]}\n')
print(f'source: {data["source"][0]}\n')
print(f'genre: {data["genre"][0]}\n')
print(f'content: {data["content"][0]}\n')
print(f'level: {data["level"][0]}')

title: salud 2021 06 04

source: https://www.elmundo.es/

genre: news-article-news-article-ciencia-y-salud

content: Más de 10 millones de personas vacunadas contra la Covid-19. Un hito positivo y simbólico en la lucha contra la pandemia de coronavirus, que sigue llevándose vidas. 97 fallecimientos se han notificado en las últimas 24 horas, según los datos difundidos este viernes por Sanidad.            En el plano político, Sanidad ha enviado esta tarde a las comunidades la orden con las nuevas restricciones sobre el ocio nocturno y la hostelería recordándoles que son de obligado cumplimiento y este sábado se publicarán en el BOE. A las comunidades que han anunciado su voluntad de desobedecerlas les cabe aún el recurso ante el propio ministerio y ante la Audiencia Nacional.            Los datos del coronavirus en EspañaCifras totales: 3.697.987 casos de coronavirus confirmados con prueba diagnóstica de infección activa; ha habido 80.196 muertos con test positivo a 4 de junio.590.684 e

In [204]:
#print(html_elmundo.prettify())
# This piece of code is for debugging purpose only - please ignore. 
url = 'https://www.elmundo.es/'
f = urlopen(url)
html_elmundo = BeautifulSoup(f, 'lxml')

for node in html_elmundo.find_all("a"):
    if len(node.attrs.keys()) == 1 and 'href' in node.attrs.keys() and node.attrs['href'].startswith('https:') and node.attrs['href'].endswith('.html'):
        address = node.attrs['href'].split('/')
        genre = 'news-article-' + address[3]
        
        child_url = node.attrs['href']
        f_child = urlopen(child_url)
        html_child = BeautifulSoup(f_child, 'lxml')
        
        for node_in_child in html_child.find_all("script"):
            if 'type' in node_in_child.attrs and node_in_child.attrs['type'] == 'application/ld+json':
                for script_ in node_in_child.contents:
                    jsonStr = script_.strip()
                    jsonStr = jsonStr.replace('\n', '')    # remove newline
                    jsonList = json.loads(jsonStr)         # dictionary
                    
                    if type(jsonList) == dict and 'articleBody' in jsonList.keys():
                        print(jsonList['articleBody'], '\n')
        f_child.close()
        break
        
f.close()

Más de 10 millones de personas vacunadas contra la Covid-19. Un hito positivo y simbólico en la lucha contra la pandemia de coronavirus, que sigue llevándose vidas. 97 fallecimientos se han notificado en las últimas 24 horas, según los datos difundidos este viernes por Sanidad.            En el plano político, Sanidad ha enviado esta tarde a las comunidades la orden con las nuevas restricciones sobre el ocio nocturno y la hostelería recordándoles que son de obligado cumplimiento y este sábado se publicarán en el BOE. A las comunidades que han anunciado su voluntad de desobedecerlas les cabe aún el recurso ante el propio ministerio y ante la Audiencia Nacional.            Los datos del coronavirus en EspañaCifras totales: 3.697.987 casos de coronavirus confirmados con prueba diagnóstica de infección activa; ha habido 80.196 muertos con test positivo a 4 de junio.590.684 en Andalucía (9.927 muertos)124.819 en Aragón (3.511 muertos)52.603 en Asturias (1.970 muertos)60.582 en Baleares (839